# Semi-supervised Classification

This jupyter notebook contains a training script for the https://github.com/beresandras/semisupervised-classification-keras repository, and is intended to be used in a Google Colab environment.

In [ ]:
# comment out after first run
!git clone https://github.com/beresandras/semisupervised-classification-keras

In [ ]:
import sys
import pickle
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

sys.path.insert(0,'/content/contrastive-classification-keras')
from dataset import prepare_dataset
from augmentations import RandomResizedCrop, RandomColorJitter
from algorithms import CrossEntropy, InfoNCE, SuNCEt, PAWS
from models import KNNClassifier

In [ ]:
# hyperparameters
num_epochs = 25
width = 128
k_values = (20, 200) # for knn classification
batch_sizes = {  # unlabeled, labeled
    CrossEntropy: (250, 250),
    InfoNCE: (500, 25),
    SuNCEt: (250, 250),
    PAWS: (250, 250),
}
hyperparams = {
    CrossEntropy: {},
    InfoNCE: {"temperature": 0.1},
    SuNCEt: {"temperature": 0.1, "supervised_loss_weight": 1.0},
    PAWS: {"temperature": 0.1, "sharpening": 0.25},
}

# select an algorithm
Algorithm = SuNCEt  # CrossEntropy, InfoNCE, SuNCEt, PAWS

In [ ]:
# load STL10 dataset
train_dataset, eval_train_dataset, eval_test_dataset = prepare_dataset(
    *batch_sizes[Algorithm]
)

In [ ]:
model = Algorithm(
    augmenter=keras.Sequential(
        [
            layers.Input(shape=(96, 96, 3)),
            preprocessing.Rescaling(1 / 255),
            preprocessing.RandomFlip("horizontal"),
            RandomResizedCrop(scale=(0.2, 1.0), ratio=(3 / 4, 4 / 3)),
            RandomColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2),
        ],
        name="augmenter",
    ),
    encoder=keras.Sequential(
        [
            layers.Input(shape=(96, 96, 3)),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Flatten(),
            layers.Dense(width),
        ],
        name="encoder",
    ),
    **hyperparams[Algorithm]
)

In [ ]:
evaluators = [
    KNNClassifier(
        train_dataset=eval_train_dataset,
        test_dataset=eval_test_dataset,
        model=model,
        k=k,
    )
    for k in k_values
]
model.compile(
    optimizer=keras.optimizers.Adam(),
)

# run training
history = model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=eval_test_dataset,
    callbacks=[
        keras.callbacks.LambdaCallback(on_epoch_end=evaluator.evaluate)
        for evaluator in evaluators
    ],
)

# save history
with open("{}.pkl".format(Algorithm.__name__), "wb") as output_file:
    pickle.dump(history.history, output_file)